In [10]:
from fast_to_SQL.fast_to_SQL import to_sql_fast
import pandas as pd
import numpy as np
from urllib.parse import quote_plus
import sqlalchemy

params = quote_plus(
    """DRIVER={ODBC Driver 13 for SQL Server};
    SERVER=server-dw01.database.windows.net;
    DATABASE=DataWarehouseStaging;
    UID=dw_userlogin;
    PWD=kljDueK!7"""
)

ENGINE = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [8]:
def yyyymm_to_date(df):
    series = df['TransMth']
    series = series.astype(str)
    yr = series.str[:4]
    mth = series.str[4:]
    return yr+'-'+mth+'-01'

def relative(df, TO):
    unpivot_df = df.melt(id_vars = ['com_id_h', 'Account ID', 'Opportunity ID', 'Decision Date', 'First of Decision Month'], var_name = 'vol_month', value_name = 'volume')
    if TO == True: #append volumes from SeasonalPayoff too
        acc_df = df[['com_id_h', 'Account ID', 'Opportunity ID', 'Decision Date', 'First of Decision Month']]
        payoff_df = pd.read_sql('''SELECT * FROM dbo.SeasonalPayoffRaw''', ENGINE)
        vol_df = acc_df.merge(payoff_df, how = 'inner', left_on = 'Account ID', right_on = 'AccountId')
        vol_df = vol_df[['com_id_h', 'Account ID', 'Opportunity ID', 'Decision Date', 'First of Decision Month', 'TransMth', 'TransAmt']]
        vol_df['vol_month'] = yyyymm_to_date(vol_df)
        vol_df = vol_df.rename(columns = {'TransAmt':'volume'})
        vol_df = vol_df[['com_id_h', 'Account ID', 'Opportunity ID', 'Decision Date', 'First of Decision Month', 'vol_month', 'volume', ]]
        unpivot_df = unpivot_df.append(vol_df, ignore_index = True)
    unpivot_df['vol_month'] = pd.to_datetime(unpivot_df['vol_month'])
    unpivot_df['rel_vol_month'] = ((unpivot_df['vol_month'] - unpivot_df['First of Decision Month'])/np.timedelta64(1, 'M')).round(0).astype(int)
    pivot_df = pd.pivot_table(unpivot_df, values = 'volume', index = ['com_id_h', 'Opportunity ID', 'Decision Date', 'First of Decision Month'], columns = 'rel_vol_month')
    pivot_df = pivot_df.reset_index()
    pivot_df.columns = [str(i) for i in pivot_df.columns]
    return pivot_df 

In [5]:
TO_df = pd.read_sql(''' SELECT d.*
    ,o.[Account ID]
	,o.[Opportunity ID]
	,o.[Decision Date]
	,dbo.BoM(o.[Decision Date]) AS [First of Decision Month]

FROM pam.TO_dataset_dedupe AS d

LEFT JOIN pam.MID_dict AS m
	ON d.com_id_h = m.com_id_h

LEFT JOIN [marketing].[WP_Attri_Mailers] AS t
	ON d.com_id_h = t.Hashed_ID
		OR m.company_id = t.Company_ID

LEFT JOIN Opportunities AS o
	ON t.[Opportunity ID] = o.[Opportunity ID]

WHERE o.[Decision Date] IS NOT NULL

''', ENGINE)

In [11]:
TO_df_relative = relative(TO_df, TO = True)

In [13]:
to_sql_fast(TO_df_relative, name = 'pam.TO_dataset_relative', engine = ENGINE, if_exists = 'replace')

In [14]:
trans_df = pd.read_sql(''' SELECT d.*
    ,o.[Account ID]
	,o.[Opportunity ID]
	,o.[Decision Date]
	,dbo.BoM(o.[Decision Date]) AS [First of Decision Month]

FROM pam.trans_dataset_dedupe AS d

LEFT JOIN pam.MID_dict AS m
	ON d.com_id_h = m.com_id_h

LEFT JOIN [marketing].[WP_Attri_Mailers] AS t
	ON d.com_id_h = t.Hashed_ID
		OR m.company_id = t.Company_ID

LEFT JOIN Opportunities AS o
	ON t.[Opportunity ID] = o.[Opportunity ID]

WHERE o.[Decision Date] IS NOT NULL

''', ENGINE)

In [15]:
trans_df_relative = relative(trans_df, TO = False)

In [16]:
to_sql_fast(trans_df_relative, name = 'pam.trans_dataset_relative', engine = ENGINE, if_exists = 'replace')